In [1]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 


In [2]:
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [3]:
train  = pd.read_csv('train_E6oV3lV.csv')
test = pd.read_csv('test_tweets_anuFYb8.csv')

In [4]:
dataset = train.append(test, ignore_index=True)
dataset.shape

C:\Users\SAIKAT\AppData\Local\Temp\ipykernel_784\3091561134.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = train.append(test, ignore_index=True)


(49159, 3)

In [5]:
# Function to remove unwanted pattterns
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt    


In [6]:
# Apply remove_pattern function to remove all unwanted pattern
dataset['tidy_tweet'] = np.vectorize(remove_pattern)(dataset['tweet'], "@[\w]*")

In [7]:
dataset['tidy_tweet'] = dataset['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
dataset.head()

C:\Users\SAIKAT\AppData\Local\Temp\ipykernel_784\3139356624.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset['tidy_tweet'] = dataset['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")


,id,label,tweet,tidy_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so sel...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for #lyft credit i can t use cause th...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation,factsguide society now #motivation


In [8]:
dataset['tidy_tweet'] = dataset['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
dataset.head()

,id,label,tweet,tidy_tweet
0,1,0.0,@user when a father is dysfunctional and is s...,when father dysfunctional selfish drags kids i...
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks #lyft credit cause they offer wheelchai...
2,3,0.0,bihday your majesty,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...,#model love take with time
4,5,0.0,factsguide: society now #motivation,factsguide society #motivation


In [9]:
tokenized_tweet = dataset['tidy_tweet'].apply(lambda x: x.split())
tokenized_tweet.head()

0    [when, father, dysfunctional, selfish, drags, ...
1    [thanks, #lyft, credit, cause, they, offer, wh...
2                              [bihday, your, majesty]
3                     [#model, love, take, with, time]
4                   [factsguide, society, #motivation]
Name: tidy_tweet, dtype: object

In [10]:
#Stemming
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet_porterstemmer = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenized_tweet_porterstemmer.head()


0    [when, father, dysfunct, selfish, drag, kid, i...
1    [thank, #lyft, credit, caus, they, offer, whee...
2                              [bihday, your, majesti]
3                     [#model, love, take, with, time]
4                         [factsguid, societi, #motiv]
Name: tidy_tweet, dtype: object

In [11]:
from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()
tokenized_tweet_lemmatize = tokenized_tweet.apply(lambda x:[lemmatizer.lemmatize(i) for i in x])

LookupError: 
**********************************************************************
  Resource [93momw-1.4[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('omw-1.4')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/omw-1.4[0m

  Searched in:
    - 'C:\\Users\\SAIKAT/nltk_data'
    - 'E:\\Django\\newChat\\env\\nltk_data'
    - 'E:\\Django\\newChat\\env\\share\\nltk_data'
    - 'E:\\Django\\newChat\\env\\lib\\nltk_data'
    - 'C:\\Users\\SAIKAT\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
for i in range(len(tokenized_tweet_porterstemmer)):
    tokenized_tweet_porterstemmer[i] = ' '.join(tokenized_tweet_porterstemmer[i])

In [ ]:
# Lemmatization
for i in range(len(tokenized_tweet_lemmatize)):
    tokenized_tweet_lemmatize[i] = ' '.join(tokenized_tweet_lemmatize[i])

In [ ]:
dataset['tidy_tweet'] = tokenized_tweet_lemmatize

In [ ]:
dataset.head()

In [ ]:
# all_words = ' '.join([text for text in dataset['tidy_tweet']])
# from wordcloud import WordCloud
# wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

# plt.figure(figsize=(10, 7))
# plt.imshow(wordcloud, interpolation="bilinear")
# plt.axis('off')
# plt.show()


In [ ]:
def hashtag_extract(x):
    hashtags = []
    # Loop over the words in the tweet
    for i in x:
        ht = re.findall(r"#(\w+)", i)
        hashtags.append(ht)

    return hashtags


In [ ]:
# extracting hashtags from non racist/sexist tweets

HT_regular = hashtag_extract(dataset['tidy_tweet'][dataset['label'] == 0])

# extracting hashtags from racist/sexist tweets
HT_negative = hashtag_extract(dataset['tidy_tweet'][dataset['label'] == 1])

# unnesting list
HT_regular = sum(HT_regular,[])
HT_negative = sum(HT_negative,[])


In [ ]:
# List Of Positive hashtag
a = nltk.FreqDist(HT_regular)
d = pd.DataFrame({'Hashtag': list(a.keys()),
                  'Count': list(a.values())})
# selecting top 10 most frequent hashtags     
d = d.nlargest(columns="Count", n = 10) 
plt.figure(figsize=(16,4))
ax = sns.barplot(data=d, x= "Hashtag", y = "Count")
ax.set(ylabel = 'Count')
plt.show()


In [ ]:

b = nltk.FreqDist(HT_negative)
e = pd.DataFrame({'Hashtag': list(b.keys()), 'Count': list(b.values())})
# selecting top 10 most frequent hashtags
e = e.nlargest(columns="Count", n = 10)   
plt.figure(figsize=(16,5))
ax = sns.barplot(data=e, x= "Hashtag", y = "Count")
ax.set(ylabel = 'Count')
plt.show()


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(dataset['tidy_tweet'])


In [ ]:
train_bow = bow[:24579,:]
test_bow = bow[24579:,:]


In [ ]:
# tidy_tweet = pd.get_dummies(dataset['tidy_tweet'], drop_first = True)
# x = np.asarray(tidy_tweet).reshape((49159,))
# x.shape

In [ ]:
label = pd.get_dummies(dataset['label'], drop_first = True)
y = np.asarray(label).reshape((49159,))
y.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
# dataset['tidy_tweet']
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(bow, y, random_state=42, test_size=0.3)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(xtrain_bow, ytrain)

In [ ]:
pred = model.predict(xvalid_bow)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
confusion_matrix(yvalid, pred)

In [ ]:
accuracy_score(yvalid, pred) * 100

In [ ]:
import pickle
# pickle.dump(cv, open('CountVectorizer.pkl', 'wb'))
pickle.dump(model, open('spam_classifier.pkl', 'wb'))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize = (5,5))
sns.heatmap(confusion_matrix(yvalid, pred), annot = True)